# Goal of notebook:
- convert all types of book identifiers (sbn, isbn10, isbn13) to a valid isbn 13
- validity of isbn13 and isbn 10 checked with checksum (its last digit)
- explore isbnlib library: isbn conversion, metadata fetching, adding additional sources
- manual implementation of isbn conversions (not used, for testing purposes)

In [4]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD
import isbnlib
import sys
sys.path.append('../')

sys.path.append('../andre')
import andre.utils as utils
from andre.utils import schema as SCHEMA
import collections

In [6]:
graph_bnf = Graph()
graph_bnf.parse("final_datasets/bnf.ttl", format="turtle")

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x0000023FA6E74860>
Traceback (most recent call last):
  File "C:\Users\dre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\dre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\isodate\isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: 'Port- d-es'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x0000023FA6E74860>
Traceback (most recent call last)

<Graph identifier=N0d44e3c1be9344e3823e2687b54f2002 (<class 'rdflib.graph.Graph'>)>

In [14]:
isbnlib.isbn_from_words("Pousse-poussette")

'9782211012157'

In [15]:
isbnlib.meta("9782211012157", service='openlopenl')

{'ISBN-13': '9782211012157',
 'Title': 'Pousse-Poussette',
 'Authors': ['Michel Gay'],
 'Publisher': "L'Ecole des loisirs",
 'Year': '1982',
 'Language': ''}

In [7]:
def sbn_to_isbn10(sbn):
    # Check if the input is a valid SBN string
    if len(sbn) == 9 and sbn.isdigit():
        return "0" + sbn

for bnf_book in graph_bnf.subjects(RDF.type, utils.schema.Book):
    book_data = utils.extract_data_bnf(graph_bnf, bnf_book)
    isbn = isbnlib.canonical(book_data.isbn)
    ean = isbnlib.canonical(book_data.ean) # ean is supposed to be 13 digits
    if len(isbn) == 9:
        isbn = sbn_to_isbn10(isbn)
    
    valid_isbn13 = isbnlib.to_isbn13(isbn)
    valid_ean13 = isbnlib.to_isbn13(ean) # if valid_ean13 exists and valid should equals to original EAN

    if valid_isbn13 and valid_ean13 and valid_isbn13 != valid_ean13: # put both in graph because likely to be different editions but difficult to choose between the right one
        # print(isbn, valid_isbn13, valid_ean13)
        graph_bnf.add((bnf_book, utils.pbs.validISBN13, Literal(valid_isbn13)))
        graph_bnf.add((bnf_book, utils.pbs.validISBN13, Literal(valid_ean13))) 
    elif valid_isbn13 and valid_ean13 and valid_isbn13 == valid_ean13:
        graph_bnf.add((bnf_book, utils.pbs.validISBN13, Literal(valid_isbn13)))
    elif valid_isbn13 and not valid_ean13:
        graph_bnf.add((bnf_book, utils.pbs.validISBN13, Literal(valid_isbn13)))
    elif not valid_isbn13 and valid_ean13:
        graph_bnf.add((bnf_book, utils.pbs.validISBN13, Literal(valid_ean13)))
    else:
        print("no valid isbn", book_data.isbn) # don' t add anything: if empty value, graphdb will match with other empty value
        # graph_bnf.add((bnf_book, utils.pbs.validISBN13, Literal("")))

 


no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 286357008
no valid isbn 2081617074
no valid isbn 
no valid isbn 221107952
no valid isbn 22110662941
no valid isbn 
no valid isbn 2211012152
no valid isbn 2211016832
no valid isbn 2211026953
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 2904292424
no valid isbn 2878330885
no valid isbn 
no valid isbn 29092190504
no valid isbn 221102684X
no valid isbn 28696102830
no valid isbn 
no valid isbn 2226070696
no valid isbn 2910635685


no valid isbn 
no valid isbn 2868770941
no valid isbn 
no valid isbn 28673879313
no valid isbn 220314001X
no valid isbn 2711836810
no valid isbn 2211050074
no valid isbn 2226101980
no valid isbn 2081644939
no valid isbn 2070310344
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 2211064231
no valid isbn 2748500381
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn ISSN 16397525
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 2748502039
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 
no valid isbn 3505371133472
no valid isbn 3505371133465
no valid i

In [4]:
fichier = open("../final_datasets/bnf_cleanISBN13.ttl","wb")
graph_bnf.serialize(fichier) 
fichier.close()

In [3]:
isbnlib.to_isbn13("2246002311")

'9782246002314'

In [5]:
graph_constellations = Graph()
graph_constellations.parse("final_datasets/constellations_NoDuplicatePublisher.ttl", format="turtle")

<Graph identifier=N4bba4aafa0c84487b51493ada95bef83 (<class 'rdflib.graph.Graph'>)>

In [6]:
for constellations_book in graph_constellations.subjects(RDF.type, utils.schema.Book):
    book_data = utils.extract_data_bnf(graph_constellations, constellations_book)
    isbn = book_data.isbn

    if len(isbn) == 9:
        isbn = sbn_to_isbn10(isbn)
    
    valid_isbn13 = isbnlib.to_isbn13(isbn)
    if valid_isbn13:
        graph_constellations.add((constellations_book, utils.pbs.validISBN13, Literal(valid_isbn13)))
    else:
        print(" no valid isbn")



 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn
 no valid isbn


In [7]:
fichier = open("../final_datasets/constellations_cleanISBN13.ttl","wb")
graph_constellations.serialize(fichier) 
fichier.close()

# tests metadata fetching of isbnlib

In [5]:
isbnlib.meta("2012006043", service='bnf')

{'ISBN-13': '9782012006041',
 'Title': 'Tu ne voleras pas',
 'Authors': ['Donner, Christophe'],
 'Publisher': 'Hachette Jeunesse (Paris)',
 'Year': '2000',
 'Language': 'fre'}

In [3]:
isbnlib.meta("9782383490456", service = 'bnf')

{'ISBN-13': '9782383490456',
 'Title': 'Kidnapping à Bombay',
 'Authors': ['Pendule, Roseline'],
 'Publisher': 'Gulf stream éditeur (Nantes)',
 'Year': '2023',
 'Language': 'fre'}

In [4]:
isbnlib.meta("9782203242111", service = 'bnf')

{'ISBN-13': '9782203242111',
 'Title': 'Un nouveau rival',
 'Authors': ['Gray, Annelise'],
 'Publisher': 'Casterman (Bruxelles)',
 'Year': '2023',
 'Language': 'fre'}

In [12]:
isbnlib.isbn_from_words("chateau de sable")

'9782908136661'

In [14]:
isbnlib.meta("9782908136661")

{'ISBN-13': '9782908136661',
 'Title': 'Un château de sable - récit',
 'Authors': ['Jean-Paul de Sauw'],
 'Publisher': '',
 'Year': '1993',
 'Language': 'fr'}

In [16]:
isbnlib.desc("9782908136661")
# peut ameliorer desc pour pull de bnf

''

In [26]:
isbnlib.meta("9791023506297", service='bnf')

{'ISBN-13': '9791023506297',
 'Title': 'Bouh ! : le livre qui fait le plus peur du monde',
 'Authors': ['Louchard, Antonin'],
 'Publisher': 'Seuil jeunesse (Paris)',
 'Year': '2016',
 'Language': 'fre'}

In [28]:
isbnlib.desc("9791023506297")

"Dans ce nouvel album d'Antonin Louchard, le lecteur est accueilli par un\npetit fantôme bien décidé à lui faire peur. En vain. Mais il réussira tout\nde même à lui arracher un sursaut... Non ? Un frisson alors ? Toujours pas\n? A court d'idées et vexé comme un pou, le petit fantôme est sauvé par le\ngong. Car oui, les fantômes sont comme tout le monde, ils vont à l'école,\nfont leurs devoirs et doivent écouter leur maman quand elle les appelle\npour manger. Mais ce n'est que partie remise !"

In [25]:
isbnlib.cover("9791023506297")

{}

In [37]:
isbnlib.desc("2748501721")

"Ca fait cinq ans qu'ils en rêvent : à leur majorité, Maxime et Patrice\nquitteront leur banlieue et partiront pour une île. Tout est minutieusement\npréparé. Mais à quelques jours du départ, Patrice est impliqué dans une\nhistoire de vol et victime d'un chantage..."

In [10]:
isbnlib.classify("9782012006041")

ISBNLibHTTPError: an HTTP error has ocurred (403 Are you making many requests?)

In [40]:
isbnlib.cover("2748501721")

{}

In [41]:
isbnlib.doi2tex("2748501721")

ISBNLibHTTPError: an HTTP error has ocurred ((404) Not Found)

In [22]:
isbn10_to_isbn13("274041515X")

'9782740415153'

# isbn 9/10 to isbn 13 (not used, coded for educational purpose)

In [57]:
def compute_isbn10_checksum(isbn10):
    checksum = 0
    for i in range(9):
        digit = int(isbn10[i])
        checksum += digit *(i+1)
    checksum = checksum % 11
    if checksum == 10:
        checksum = 'X'
    return str(checksum)

compute_isbn10_checksum("2910635685")

'6'

In [56]:
compute_isbn10_checksum("2748501721")

'1'

In [8]:


def compute_isbn13_checksum(isbn13):
    checksum = 0
    for i in range(12):
        digit = int(isbn13[i])
        if i % 2 == 0:
            checksum += digit
        else:
            checksum += 3 * digit
    checksum = 10 - (checksum % 10)
    if checksum == 10:
        checksum = 0
    return str(checksum)

def sbn_to_isbn10(sbn):
    # Check if the input is a valid SBN string
    if len(sbn) != 9 or not sbn.isdigit():
        return "Invalid SBN string"

    isbn10 = "0" + sbn
    return isbn10

def is_isbn10_format_correct(isbn10):
   # Check if the input is a valid ISBN-10 string
    if len(isbn10) != 10 :
        return False
    if (isbn10[:-1].isdigit() or 
        isbn10[-1] != 'X' and not isbn10[-1].isdigit()):
        return False

def is_isbn13_format_correct(isbn13):
    if len(isbn13) != 13:
        return False
    if not isbn13.isdigit():
        return False
    if compute_isbn13_checksum(isbn13) != int(isbn13[12]):
        return False
    return True

def is_sbn_format_correct(sbn):
    if len(sbn) == 9 and sbn.isdigit():
        return True
    else: 
        return False

    
def isbn10_to_isbn13(isbn10):
    isbn13_prefix = "978"
    isbn10_without_checksum = isbn10[:-1]
    isbn13_without_checksum = isbn13_prefix + isbn10_without_checksum
    isbn13 = isbn13_without_checksum + checksum
    return isbn13

In [26]:
print(isbnlib.is_isbn10("0746048319"))
print(isbnlib.is_isbn13("0746048319"))
print(isbnlib.to_isbn13("0746048319"))

True
False
9780746048313


In [30]:
print(isbnlib.to_isbn13("2748501721"))

9782748501728
